In [31]:
def create_load_save_dir(source):
    try:
        Path.mkdir(Path.cwd() / f'saved_{source}_csv')
    except:
        pass
    save_dir = (Path.cwd() / f'saved_{source}_csv').as_posix()
    return save_dir

In [32]:
local_save_dir_cian = create_load_save_dir('cian')

In [33]:
def get_saved_files_names(source):
    try:
        with open(f'uploaded_files_{source}.txt', 'r') as uploaded_txt:
            list_loaded_files = uploaded_txt.read()
        uploaded_txt.close()
        list_loaded_files = list_loaded_files.split('\n')[:-1]
    except:
        list_loaded_files = []
    return list_loaded_files

In [34]:
handled_files_cian = get_saved_files_names('cian')

In [37]:
def download_local_yadisk_files(yandex_api_token, handled_files, save_dir):
    y = yadisk.YaDisk(token=yandex_api_token)
    if y.check_token():
        print('yandex token valid')
        disk_files_info = list(y.listdir("/cian"))
        disk_files_routes = [disk_files_info[i].path for i in range(len(disk_files_info))]
        saved_files = []
        cnt = 0
        for file in disk_files_routes:
            if Path(file).stem not in handled_files:
                y.download(file, save_dir + '/' + Path(file).name)
                saved_files.append(Path(file).name)
                cnt += 1
        print('Успешно загружено {} файлов с личного диска: {}'.format(cnt, saved_files))
        is_error = False
        return saved_files, is_error
    else:
        print('yandex token invalid')
        print(
            'перейдите по ссылке: https://oauth.yandex.ru/authorize?response_type=token&client_id=<> вставив свой айди приложения')
        print('id приложения можно получить в: https://oauth.yandex.ru/ - пользователь yanedvizhkatop')
        print('мануал: https://ramziv.com/article/8')
        saved_files = []
        is_error = False
        return saved_files, is_error

In [38]:
files_to_process_cian, error_file_loading_cian = download_local_yadisk_files(ya_token,handled_files_cian, local_save_dir_cian)

yandex token valid
Успешно загружено 3 файлов с личного диска: ['20-03-23.csv', '21-03-23.csv', '22-03-23.csv']


In [39]:
files_to_process_cian

['20-03-23.csv', '21-03-23.csv', '22-03-23.csv']

In [40]:
def write_saved_file_names(filename, source):
    with open(f'uploaded_files_{source}.txt', 'a+') as uploaded_txt:
        uploaded_txt.write(f"{filename}\n")
    uploaded_txt.close()

In [42]:
write_saved_file_names(Path(files_to_process_cian[1]).stem, 'cian')

# check

In [6]:
def create_load_save_dir(source):
    try:
        Path.mkdir(Path.cwd() / f'saved_{source}_csv')
    except:
        pass
    save_dir = (Path.cwd() / f'saved_{source}_csv').as_posix()
    return save_dir

In [51]:
def get_local_files(save_dir):
    p = Path(save_dir).glob('**/*')
    saved_files_list = [x.as_posix() for x in p if x.is_file()]
    return saved_files_list

In [7]:
def get_direct_link(yandex_api, sharing_link):
    pk_request = requests.get(yandex_api.format(sharing_link))
    # None если не удается преоброазовать ссылку
    return pk_request.json().get('href')

In [8]:
def get_saved_files_names(source):
    try:
        with open(f'uploaded_files_{source}.txt', 'r') as uploaded_txt:
            list_loaded_files = uploaded_txt.read()
        uploaded_txt.close()
        list_loaded_files = list_loaded_files.split('\n')[:-1]
    except:
        list_loaded_files = []
    return list_loaded_files

In [9]:
def download_yadisk_files(yandex_api, sharing_link, list_file_exist, save_dir):
    direct_link = get_direct_link(yandex_api, sharing_link)
    try:
        saved_files = []
        if direct_link:
            download = requests.get(direct_link)
            zips = zipfile.ZipFile(io.BytesIO(download.content))
            cnt = 0
            for member in zips.namelist():
                filename = os.path.basename(member)
                if not filename or Path(filename).stem in list_file_exist or 'циан' in filename:
                    continue
                src = zips.open(member)
                target = open(os.path.join(save_dir, filename), 'wb')
                with src, target:
                    shutil.copyfileobj(src, target)
                    cnt += 1
                    saved_files.append(filename)
                target.close()
            print('Succesfully downloaded {} files from "{}"'.format(cnt, sharing_link))
            is_error = False
            return saved_files, is_error
        else:
            print('Failed to download files from "{}"'.format(sharing_link))
            saved_files = []
            is_error = True
            return saved_files, is_error
    except Exception as exc:
        print(exc)
        saved_files = []
        is_error = True
        return saved_files, is_error

In [12]:
local_save_dir_avito = create_load_save_dir('avito')

In [13]:
handled_files_avito = get_saved_files_names('avito')

In [475]:
files_to_process_avito, error_file_loading_avito = download_yadisk_files(ya_api, ya_link,
                                                                                     handled_files_avito, 
                                                                                     local_save_dir_avito)

Succesfully downloaded 1 files from "https://disk.yandex.ru/d/sVgz0GiqjhCsNg"


In [476]:
files_to_process_avito

['20-03-23 (без дублей).csv']

In [1]:
files_to_process_avito = ['20-03-23 (без дублей).csv']

In [15]:
files_to_process_avito

['20-03-23 (без дублей).csv']

In [222]:
def get_tables_info(engine):
    cities_query = 'SELECT * FROM city'
    types_query = 'SELECT * FROM realty_types'
    source_query = 'SELECT id, name FROM sources'
    district_query = 'SELECT DISTINCT id, city_id, name FROM districts'
    try:
        con_obj = engine.connect()
        city_db = pd.read_sql(text(cities_query), con=con_obj)
        realty_type_db = pd.read_sql(text(types_query), con=con_obj)
        source_db = pd.read_sql(text(source_query), con=con_obj)
        district_db = pd.read_sql(text(district_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        realty_type_db = None
        source_db = None
        district_db = None
        exc_code = exc.code
    return city_db, realty_type_db, source_db, district_db, exc_code


def get_version(engine, today_date, source_id):
    last_version = f"SELECT Max(version) FROM realty WHERE date < '{today_date}' AND source_id = {source_id}"
    try:
        con_obj = engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        last_version_db = None
        exc_code = exc.code
    return last_version_db, exc_code


def source_id_from_link(link_value, source_dict):
    try:
        return list(source_dict.keys())[list(source_dict.values()).index(link_value.split('.')[1])]
    except:
        return None


def city_id_from_link(city_value, city_dict):
    try:
        return int(list(city_dict.keys())[list(city_dict.values()).index(city_value)])
    except Exception as e:
        return None


def type_id_from_room(room_value, type_dict):
    try:
        for ind, values in type_dict.items():
            if room_value in values:
                tid = ind
                return tid
        if any(elem in room_value for elem in ['Доля', 'Аукцион']):
            # tid = 9 # аукцион
            return None
        elif all(elem in room_value for elem in ['Комната ', ' в ', '-к']):
            tid = 1  # комната в квартире
            return tid
        elif any(elem in room_value for elem in
                 ['6-к.', '7-к.', '8-к.', '9-к.', '10 и более-к.', '10 комнат и больше']):
            tid = 8  # 6 комнат и более
            return tid
        else:
            return None  # none
    except:
        return None


# создание поля addr из данных avito
# поле должно совпасть с форматом cian
# формат: Томская область; Томск; р-н Кировский; мкр. Преображенский; улица Дзержинского; 36
def avito_addr_from_row(row, city_df):
    # добавление города для совпадения форматов
    try:
        addr = city_df[city_df.url_avito == row['city']]['name'].values[0]
        addr += '; '
    except:
        addr = ''
    # добавляем район, в авито он может быть None
    try:
        if len(row['Район']) == 0:
            pass
        else:
            addr += row['Район']
            addr += '; '
    except:
        pass
    # добавляем улицу, дом, пр. в авито он может быть None
    try:
        for value in row['Улица'].split(', '):
            addr += value
            addr += '; '
    except:
        pass
    # drop duplicates
    addr = addr[:-2]
    if len(addr.split('; ')) != len(set(addr.split('; '))):
        addr = '; '.join(list(dict.fromkeys(addr.split('; '))))
    return addr


def square_from_ploshad(square_value):
    try:
        return float(square_value[:-3].replace(',', '.'))
    except:
        return None


def floor_floors_from_etazh(etazh_value):
    try:
        return [etazh_value.split('/')[0], etazh_value.split('/')[1].replace(' эт.', '')]
    except:
        return [None, None]


def get_date_from_name(fname):
    try:
        return datetime.strptime(fname.replace("(без дублей)", "").replace("циан", "").replace(" ", ""), "%d-%m-%y")
    except:
        return datetime.strptime(Path(fname).stem.replace("(без дублей)", "").replace("циан", "").replace(" ", ""), "%d-%m-%y")

In [223]:
def create_realty(df, fname, sql_engine, source):
    if source == 'avito':
        try:
            # сбор данных из таблиц в бд
            city_df, realty_type_df, source_df, district_df, exc_code = get_tables_info(sql_engine)

            # удаление корявых данных, дубликатов
            index_to_drop = df[
                (df['Площадь'].isna()) | (df['Заголовок'] == df['Комнат']) | (df['Цена'].isna())].index.tolist()
            if len(index_to_drop) != 0:
                df.drop(index_to_drop, inplace=True)
            df.drop_duplicates(subset=['Ссылка'], keep='last', inplace=True)
            df.reset_index(drop=True, inplace=True)

            # city
            df['city'] = df['Ссылка'].apply(lambda x: x.split('/')[3])

            # source_id
            source_df_dict = source_df.set_index('id').to_dict()['name']
            df['source_id'] = df['Ссылка'].apply(lambda x: source_id_from_link(x, source_df_dict))

            # ad_id
            df['ad_id'] = df['Ссылка'].apply(lambda x: x.split('_')[-1])

            # city_id
            city_df_dict = city_df.set_index('id').to_dict()['url_avito']
            df['city_id'] = df['city'].apply(lambda x: city_id_from_link(x, city_df_dict))
            df.dropna(subset=['city_id'], inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.city_id = df.city_id.astype(int)

            # district_id
            df['district_id'] = None

            # type_id
            dict_realty_type = {
                '1': ['Комната'],
                '2': ['Квартира-студия', 'Апартаменты-студия', 'Гостинка', 'Студия'],
                '3': ['1-к. квартира', '1-к. апартаменты', 'Своб. планировка', '1-комнатная', '1 комн',
                      'Свободная планировка'],
                '4': ['2-к. квартира', '2-к. апартаменты', '2-комнатная', '2 комн'],
                '5': ['3-к. квартира', '3-к. апартаменты', '3-комнатная', '3 комн'],
                '6': ['4-к. квартира', '4-к. апартаменты', '4-комнатная', '4 комн'],
                '7': ['5-к. квартира', '5-к. апартаменты', '5-комнатная', '5 комн'],
                '8': ["6 комнат и более", '6 комн']
                # 9 аукцион, доля
            }
            df['type_id'] = df['Комнат'].apply(lambda x: type_id_from_room(x, dict_realty_type))
            # удалить записи аукционов и других объяв где не опредлен type_id, преобразовать цены в int
            df.dropna(subset=['type_id'], inplace=True)
            df.reset_index(drop=True, inplace=True)
            df['Цена'] = df['Цена'].astype(float)

            # addr
            df['addr'] = df.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)

            # square
            df['square'] = df['Площадь'].apply(lambda x: square_from_ploshad(x))

            # floor
            df['floor'], df['house_floors'] = zip(*df['Этаж'].apply(floor_floors_from_etazh))

            # link
            df['link'] = df['Ссылка']

            # date
            df['date'] = get_date_from_name(fname)

            # status
            df['status'] = None

            # version
            current_source = df.source_id.unique()[0]
            current_date = datetime.now().date()
            last_version = get_version(sql_engine, current_date, current_source)[0].iloc[0, 0]
            if last_version == None:
                current_version = 1
            else:
                current_version = last_version + 1

            df['version'] = current_version

            # offer_from
            df['offer_from'] = df['Тип продавца']

            # status_newa
            df['status_new'] = 1

            realty_avito = df[['source_id', 'ad_id', 'city_id', 'district_id', 'type_id', 'addr', 'square', 'floor',
                               'house_floors', 'link', 'date', 'status', 'version', 'offer_from', 'status_new', 'Цена', 'Цена за м2']]
            error_process_realty = False
        except Exception as ex:
            print(traceback.format_exc())
            realty_avito = pd.DataFrame()
            error_process_realty = True
        return realty_avito, error_process_realty

In [224]:
def process_realty(local_dir, file_to_process, sql_engine, source):
    saved_files = get_local_files(local_dir)
    realty_df = pd.DataFrame()
    for f_ind in range(len(saved_files)):
        if Path(saved_files[f_ind]).name == file_to_process:
            try:
                if source == 'avito':
                    saved_file = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
                    realty_df, error_realty_process = create_realty(saved_file, Path(saved_files[f_ind]).stem,
                                                                    sql_engine, 'avito')
                elif source == 'cian':
                    saved_file = pd.read_csv(saved_files[f_ind], delimiter=',', encoding='cp1251')
                    realty_df, error_realty_process = create_realty(saved_file, Path(saved_files[f_ind]).stem,
                                                                    sql_engine, 'cian')
            except Exception as ex:
                print(traceback.format_exc())
                print('Не удалось прочитать файл', Path(saved_files[f_ind]))
        else:
            pass
    if len(realty_df) != 0:
        print('Успешно обработан файл {}'.format(file_to_process))
        return realty_df
    else:
        print('Файл {} не удалось обработать'.format(file_to_process))
        return realty_df

In [187]:
test_1 = process_realty(local_save_dir_avito, files_to_process_avito[0], sql_engine, 'avito')

Успешно обработан файл 20-03-23 (без дублей).csv


In [188]:
test_1

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
0,3,2814505758,17,None,3,Томск; р-н Кировский; ул. Карпова; 23,17.4,5,5,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,2023-03-20,None,1,Частное лицо,1,2200000.0,126437
1,3,2856523926,17,None,4,Томск; р-н Октябрьский; ул. Дизайнеров; 3,53.5,4,10,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,None,1,Частное лицо,1,6700000.0,125234
2,3,3010508456,17,None,4,Томск; р-н Октябрьский; микрорайон Солнечный; ...,63.0,17,18,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,None,1,Агентство,1,6100000.0,96825
3,3,2884326186,17,None,5,Томск; р-н Советский; Сибирская ул.; 102/1,68.0,1,9,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,2023-03-20,None,1,Агентство,1,5100000.0,75000
4,3,2931555827,17,None,2,Томск; пос. Зональная Станция; микрорайон Южны...,20.7,9,10,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,2023-03-20,None,1,Частное лицо,1,2650000.0,128019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114152,3,2326231151,2,None,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,15,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,5882357.0,110363
114153,3,2358537090,2,None,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,20,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,5882201.0,110360
114154,3,2326806083,2,None,4,Краснодар; р-н Прикубанский; ул. Адмирала Сере...,65.1,7,19,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,6184500.0,95000
114155,3,2816012211,2,None,4,Краснодар; р-н Прикубанский; ул. им. Кирилла Р...,64.2,15,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Агентство,1,6169625.0,96100


In [225]:
def get_realty_ids(engine, today_date, source_id):
    id_query = f"SELECT link,id FROM realty WHERE date = '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        id_db = pd.read_sql(text(id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        id_db = None
        exc_code = exc.code
    return id_db, exc_code


def create_prices(df, filename, sql_engine, source):
    try:
        if source == 'avito':
            prices_df = pd.DataFrame()

            # add realty_id
            current_source_id = a.source_id.unique()[0]
            get_realty_ids(sql_engine, get_date_from_name(filename), current_source_id)
            realty_id_df = get_realty_ids(sql_engine, current_date, current_source)[0]
            prices_df['realty_id'] = df['link'].map(id_df.set_index('link')['id'])

            # price
            prices_df['price'] = df['Цена']

            # price_sqrm
            prices_df['price_sqrm'] = df['Цена за м2'].astype(float)

            # date
            file_date = get_date_from_name(filename)
            prices_df['date'] = file_date
        else: # source == 'cian':
            prices_df = pd.DataFrame()
            pass
    except Exception as ex:
        prices_df = pd.DataFrame()
        print(traceback.format_exc())
        print('Не удалось обработать для prices файл', Path(saved_files[f_ind]))


    if len(prices_df) != 0:
        print('Успешно обработан файл {} для prices'.format(filename))
        error_status = False
        return prices_df, error_status
    else:
        error_status = True
        print('Файл {} не удалось обработать для prices'.format(filename))
        return prices_df, error_status

In [226]:
files_to_process_avito[0]

'20-03-23 (без дублей).csv'

In [227]:
df_avito_prices, error_file_processing = create_prices(test_1, files_to_process_avito[0], sql_engine, 'avito')

Успешно обработан файл 20-03-23 (без дублей).csv для prices


In [220]:
files_to_process_avito[0]

'20-03-23 (без дублей).csv'

In [228]:
df_avito_prices

,realty_id,price,price_sqrm,date
0,NaN,2200000.0,126437.0,2023-03-20
1,NaN,6700000.0,125234.0,2023-03-20
2,NaN,6100000.0,96825.0,2023-03-20
3,NaN,5100000.0,75000.0,2023-03-20
4,NaN,2650000.0,128019.0,2023-03-20
...,...,...,...,...
114152,NaN,5882357.0,110363.0,2023-03-20
114153,NaN,5882201.0,110360.0,2023-03-20
114154,NaN,6184500.0,95000.0,2023-03-20
114155,NaN,6169625.0,96100.0,2023-03-20


2023-03-22 16:00:17,720| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [17]:
pd.read_csv('saved_avito_csv/20-03-23 (без дублей).csv', encoding='utf8', delimiter=';')

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
0,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 17,4 м², 5/5 эт.",1-к. квартира,"17,4 м²",5/5 эт.,2200000,126437,NaN,р-н Кировский,"ул. Карпова, 23",2 часа назад,Частное лицо,NaN,NaN,NaN
1,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 53,5 м², 4/10 эт.",2-к. квартира,"53,5 м²",4/10 эт.,6700000,125234,NaN,р-н Октябрьский,"ул. Дизайнеров, 3",26 минут назад,Частное лицо,NaN,NaN,NaN
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 63 м², 17/18 эт.",2-к. квартира,63 м²,17/18 эт.,6100000,96825,NaN,р-н Октябрьский,"микрорайон Солнечный, ул. Герасименко, 3/16",1 час назад,Агентство,Алёна,https://www.avito.ru/user/a09c2534dd270499116e...,95
3,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,"3-к. квартира, 68 м², 1/9 эт.",3-к. квартира,68 м²,1/9 эт.,5100000,75000,NaN,р-н Советский,"Сибирская ул., 102/1",1 час назад,Агентство,Галина,https://www.avito.ru/user/e8f384adaa1ac950015f...,59
4,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,"Квартира-студия, 20,7 м², 9/10 эт.",Квартира-студия,"20,7 м²",9/10 эт.,2650000,128019,NaN,NaN,"пос. Зональная Станция, микрорайон Южные Ворот...",2 часа назад,Частное лицо,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115810,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 65,1 м², 7/19 эт.",2-к. квартира,"65,1 м²",7/19 эт.,6184500,95000,Группа компаний ИНСИТИ,р-н Прикубанский,"ул. Адмирала Серебрякова, д. 3, корп. 1",5 дней назад,Застройщик,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916
115811,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 15/16 эт.",2-к. квартира,"64,2 м²",15/16 эт.,6169625,96100,Время Группа Компаний,р-н Прикубанский,"ул. им. Кирилла Россинского, д. 3/1",1 час назад,Агентство,Время Группа Компаний,https://www.avito.ru/user/0c771d40b5f5e02cde46...,NaN
115812,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 13/16 эт.",2-к. квартира,"64,2 м²",13/16 эт.,6169600,96100,Время Группа Компаний,р-н Прикубанский,"ул. им. Кирилла Россинского, д. 3/1",1 час назад,Агентство,Время Группа Компаний,https://www.avito.ru/user/0c771d40b5f5e02cde46...,NaN
115813,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 15/16 эт.",2-к. квартира,"64,2 м²",15/16 эт.,6169625,96100,Время Группа Компаний,р-н Прикубанский,"ул. им. Кирилла Россинского, д. 3/1",1 час назад,Агентство,Время Группа Компаний,https://www.avito.ru/user/0c771d40b5f5e02cde46...,NaN


# import modules and values

In [1]:
import requests
import zipfile
import io
import os 
import shutil
import pandas as pd
import logging
import configparser
import shutil
import random
import re
import traceback
import yadisk

from pathlib import Path
from datetime import datetime
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine, text, NullPool

import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_test']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

# define saving dir

In [5]:
def create_load_save_dir():
    try:
        Path.mkdir(Path.cwd()/'saved_csv')
    except:
        pass
    save_dir = (Path.cwd()/'saved_csv').as_posix()
    return save_dir

In [6]:
local_save_dir = create_load_save_dir()

In [7]:
local_save_dir

'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv'

# get list of files already in database table test_avito_parsed

In [8]:
def get_today_date():
    return datetime.strftime(datetime.today(), format = "%d/%m/%Y, %H:%M:%S")

In [5]:
def get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost,
                   localhost_port, database_username, database_password, database_name):
    
    sql_server = SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=(localhost, localhost_port)
    )

    sql_server.start()
    local_port = str(sql_server.local_bind_port)
    sql_engine = create_engine('mariadb+pymysql://{}:{}@{}:{}/{}'.format(database_username, database_password,
                                                                         localhost, local_port, database_name), 
                               poolclass=NullPool)

    return sql_server, sql_engine

In [6]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

# get table of cities

In [11]:
def get_tables_info(engine):
    cities_query='SELECT * FROM city'
    types_query='SELECT * FROM realty_types'
    source_query='SELECT id, name FROM sources'
    district_query = 'SELECT DISTINCT id, city_id, name FROM districts'
    try:
        con_obj=engine.connect()
        city_db = pd.read_sql(text(cities_query), con=con_obj)
        realty_type_db = pd.read_sql(text(types_query), con=con_obj)
        source_db = pd.read_sql(text(source_query), con=con_obj)
        district_db = pd.read_sql(text(district_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        realty_type_db = None
        source_db = None
        district_db = None
        exc_code = exc.code
    return city_db, realty_type_db, source_db, district_db, exc_code

In [12]:
city_df, realty_type_df, source_df, district_df, exc_code = get_tables_info(sql_engine)

In [97]:
exc_code is None

True

In [14]:
city_df.head(2)

,id,name,url_avito,population
0,1,Владивосток,vladivostok,0
1,2,Краснодар,krasnodar,0


In [15]:
realty_type_df

,id,name,rooms
0,1,Комната,1
1,2,Гостинка,1
2,3,1-комнатная,1
3,4,2-комнатная,2
4,5,3-комнатная,3
5,6,4-комнатная,4
6,7,5-комнатная,5
7,8,6 комнат и более,6


In [16]:
source_df

,id,name
0,1,farpost
1,2,cian
2,3,avito


In [17]:
district_df

,id,city_id,name
0,1,1,"64, 71 микрорайоны"
1,2,1,Баляева
2,3,1,БАМ
3,4,1,Борисенко
4,5,1,Весенняя
...,...,...,...
498,499,18,Матвеевка
499,500,18,Южный
500,501,18,Акварельный
501,502,18,Огурцово


# load files

In [18]:
def get_direct_link(sharing_link):
    pk_request = requests.get(ya_api.format(sharing_link))
    # None если не удается преоброазовать ссылку
    return pk_request.json().get('href')

def download_yadisk_files(sharing_link, list_file_exist, save_dir):
    direct_link = get_direct_link(sharing_link)
    if direct_link:
        download = requests.get(direct_link)
        zips = zipfile.ZipFile(io.BytesIO(download.content))
        cnt = 0
        print("files loaded")
        for member in zips.namelist():
            filename = os.path.basename(member)
            if not filename or Path(filename).stem in list_file_exist:
                continue
            src = zips.open(member)
            target = open(os.path.join(save_dir, filename), 'wb')
            with src, target:
                shutil.copyfileobj(src, target)
                cnt += 1
            target.close()
        print('Succesfully downloaded {} files from "{}"'.format(cnt, sharing_link))
        return None
    else:
        print('Failed to download files from "{}"'.format(sharing_link))
        return 'error loading files'

In [19]:
%time
# error_file_loading = download_yadisk_files(ya_link, files_in_db, local_save_dir)

CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
# error_file_loading is None

# upload files to database 

In [21]:
def get_local_files(save_dir):
    p = Path(save_dir).glob('**/*')
    saved_files_list = [x.as_posix() for x in p if x.is_file()]
    return saved_files_list

def create_csv(save_directory):
    saved_files = get_local_files(save_directory)
    cnt_files = 0
    for f_ind in range(len(saved_files)):
        try:
            saved_files_new = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
            saved_files_new['file_name'] = Path(saved_files[f_ind]).stem
            parsed_csv = parsed_csv.append(saved_files_new)
        except:
            parsed_csv = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
            parsed_csv['file_name'] = Path(saved_files[f_ind]).stem
        cnt_files += 1
        
    parsed_csv.reset_index(drop=True, inplace=True)
    print('Succesfully opened {} files'.format(cnt_files))

    return parsed_csv

In [22]:
a_full = create_csv(local_save_dir)

Succesfully opened 35 files


In [23]:
a_full

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений,file_name
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 68 м², 16/17 эт.",2-к. квартира,68 м²,16/17 эт.,7999999,117647.0,NaN,р-н Советский,"Сибирская ул., 9А",4 дня назад,Частное лицо,NaN,NaN,NaN,01-03-23 (без дублей)
1,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 25,5 м², 1/4 эт.",2-к. квартира,"25,5 м²",1/4 эт.,1950000,76471.0,NaN,р-н Октябрьский,"Дальне-Ключевская ул., 66",56 минут назад,Агентство,"Агентство недвижимости ""NOVA""",https://www.avito.ru/user/da36df070cc701954bbb...,847,01-03-23 (без дублей)
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 30 м², 1/5 эт.",2-к. квартира,30 м²,1/5 эт.,1870000,62333.0,АЖ GROUP,р-н Октябрьский,"ул. Александра Угрюмова, 1/2",2 часа назад,Агентство,АЖ GROUP,https://www.avito.ru/user/e673859327eeaeee4e57...,NaN,01-03-23 (без дублей)
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 48,8 м², 8/9 эт.",2-к. квартира,"48,8 м²",8/9 эт.,5150000,105533.0,NaN,р-н Ленинский,"Ленинский район, микрорайон Каштак, ул. Ференц...",1 час назад,Частное лицо,NaN,NaN,NaN,01-03-23 (без дублей)
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 40 м², 10/12 эт.",1-к. квартира,40 м²,10/12 эт.,3300000,82500.0,NaN,р-н Октябрьский,"ул. Ивановского, 20",2 часа назад,Агентство,"Агентство недвижимости ""NOVA""",https://www.avito.ru/user/da36df070cc701954bbb...,847,01-03-23 (без дублей)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696381,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 27,7 м², 13/15 эт.",Квартира-студия,"27,7 м²",13/15 эт.,7077350,255500.0,АГАТ-Риэлт,р-н Карасунский,"ул. Уральская, д. 75/6",2 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696382,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 22,1 м², 10/24 эт.",Квартира-студия,"22,1 м²",10/24 эт.,4641000,210000.0,АГАТ-Риэлт,р-н Карасунский,"ул. Новороссийская, лит. 2",1 день назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696383,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 22,5 м², 2/24 эт.",Квартира-студия,"22,5 м²",2/24 эт.,4612500,205000.0,АГАТ-Риэлт,р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696384,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 45,4 м², 9/9 эт.",2-к. квартира,"45,4 м²",9/9 эт.,5675000,125000.0,АГАТ-Риэлт,р-н Прикубанский,"ул. Западный Обход, д. 42, корп. 2",3 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)


In [38]:
flist = get_local_files(local_save_dir)

NameError: name 'get_local_files' is not defined

In [25]:
flist

['C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/01-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/02-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/04-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/05-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/06-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-2023.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/08-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/08-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02-23 (без дублей) 2.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02

# load table avito

In [ ]:
a = pd.read_csv(flist[4], delimiter=';', encoding='utf8')

In [88]:
a = pd.read_csv('saved_avito_csv/20-03-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [89]:
index_to_drop = a[(a['Площадь'].isna()) | (a['Заголовок'] == a['Комнат']) | (a['Цена'].isna())].index.tolist()
if len(index_to_drop) != 0:
    a.drop(index_to_drop, inplace=True)
a.drop_duplicates(subset=['Ссылка'], keep='last', inplace=True)
a.reset_index(drop=True, inplace=True)

# dev realty avito

In [90]:
# add city

In [91]:
a['city'] = a['Ссылка'].apply(lambda x: x.split('/')[3])

In [92]:
# source_id

In [98]:
source_df_dict = source_df.set_index('id').to_dict()['name']

In [99]:
def source_id_from_link(link_value, source_dict):
    try:
        return list(source_dict.keys())[list(source_dict.values()).index(link_value.split('.')[1])]
    except:
        return None

In [100]:
a['source_id'] = a['Ссылка'].apply(lambda x: source_id_from_link(x, source_df_dict))

In [101]:
# ad_id

In [102]:
a['ad_id'] = a['Ссылка'].apply(lambda x: x.split('_')[-1])

In [103]:
# city_id

In [104]:
city_df_dict = city_df.set_index('id').to_dict()['url_avito']

In [105]:
def city_id_from_link(city_value, city_dict):
    try:
        return int(list(city_dict.keys())[list(city_dict.values()).index(city_value)])
    except Exception as e:
        return None

In [106]:
a['city_id'] = a['city'].apply(lambda x: city_id_from_link(x, city_df_dict))
# удалить записи с лишними городами и преобразовать в int
a.dropna(subset=['city_id'], inplace=True)
a.reset_index(drop=True, inplace=True)
a.city_id = a.city_id.astype(int)

In [107]:
# district_id будет удален из таблицы

In [108]:
a['district_id'] = None

In [109]:
# type_id

In [110]:
dict_realty_type = {
    '1': ['Комната'],
    '2': ['Квартира-студия', 'Апартаменты-студия', 'Гостинка', 'Студия'],
    '3': ['1-к. квартира', '1-к. апартаменты', 'Своб. планировка', '1-комнатная', '1 комн', 'Свободная планировка'],
    '4': ['2-к. квартира', '2-к. апартаменты', '2-комнатная', '2 комн'],
    '5': ['3-к. квартира', '3-к. апартаменты', '3-комнатная', '3 комн'],
    '6': ['4-к. квартира', '4-к. апартаменты', '4-комнатная', '4 комн'],
    '7': ['5-к. квартира', '5-к. апартаменты', '5-комнатная', '5 комн'],
    '8': ["6 комнат и более", '6 комн'],
}
# 9 аукцион, доля

In [111]:
def type_id_from_room(room_value, type_dict):
    tid = None
    try:
        for ind, values in type_dict.items():
            if room_value in values:
                tid = ind
                return tid
        if any(elem in room_value for elem in ['Доля', 'Аукцион']):
            # tid = 9 # аукцион
            return None
        elif all(elem in room_value for elem in ['Комната ', ' в ', '-к']):
            tid = 1 # комната в квартире
            return tid
        elif any(elem in room_value for elem in ['6-к.', '7-к.', '8-к.', '9-к.', '10 и более-к.', '10 комнат и больше']):
            tid = 8 # 6 комнат и более
            return tid
        else:
            return None # none
    except:
        return None

In [129]:
a['type_id'] = a['Комнат'].apply(lambda x: type_id_from_room(x, dict_realty_type))
# удалить записи аукционов и других объяв где не опредлен type_id, преобразовать цены в int
a.dropna(subset=['type_id'], inplace=True)
a.reset_index(drop=True, inplace=True)
a['Цена'] = a['Цена'].astype(float)

In [130]:
a['asd'] = a['Цена'].apply(lambda x: empa(x))

In [131]:
a

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,...,Продавец,Ссылка на продавца,Завершённых объявлений,city,source_id,ad_id,city_id,district_id,type_id,asd
0,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 17,4 м², 5/5 эт.",1-к. квартира,"17,4 м²",5/5 эт.,2200000.0,126437,NaN,р-н Кировский,"ул. Карпова, 23",...,NaN,NaN,NaN,tomsk,3,2814505758,17,None,3,2200000
1,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 53,5 м², 4/10 эт.",2-к. квартира,"53,5 м²",4/10 эт.,6700000.0,125234,NaN,р-н Октябрьский,"ул. Дизайнеров, 3",...,NaN,NaN,NaN,tomsk,3,2856523926,17,None,4,6700000
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 63 м², 17/18 эт.",2-к. квартира,63 м²,17/18 эт.,6100000.0,96825,NaN,р-н Октябрьский,"микрорайон Солнечный, ул. Герасименко, 3/16",...,Алёна,https://www.avito.ru/user/a09c2534dd270499116e...,95,tomsk,3,3010508456,17,None,4,6100000
3,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,"3-к. квартира, 68 м², 1/9 эт.",3-к. квартира,68 м²,1/9 эт.,5100000.0,75000,NaN,р-н Советский,"Сибирская ул., 102/1",...,Галина,https://www.avito.ru/user/e8f384adaa1ac950015f...,59,tomsk,3,2884326186,17,None,5,5100000
4,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,"Квартира-студия, 20,7 м², 9/10 эт.",Квартира-студия,"20,7 м²",9/10 эт.,2650000.0,128019,NaN,NaN,"пос. Зональная Станция, микрорайон Южные Ворот...",...,NaN,NaN,NaN,tomsk,3,2931555827,17,None,2,2650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114152,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 53,3 м², 15/24 эт.",2-к. квартира,"53,3 м²",15/24 эт.,5882357.0,110363,Группа компаний ИНСИТИ,р-н Прикубанский,"б-р им. Адмирала Пустошкина, д. 11, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2326231151,2,None,4,5882357
114153,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 53,3 м², 20/24 эт.",2-к. квартира,"53,3 м²",20/24 эт.,5882201.0,110360,Группа компаний ИНСИТИ,р-н Прикубанский,"б-р им. Адмирала Пустошкина, д. 11, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2358537090,2,None,4,5882201
114154,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 65,1 м², 7/19 эт.",2-к. квартира,"65,1 м²",7/19 эт.,6184500.0,95000,Группа компаний ИНСИТИ,р-н Прикубанский,"ул. Адмирала Серебрякова, д. 3, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2326806083,2,None,4,6184500
114155,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 15/16 эт.",2-к. квартира,"64,2 м²",15/16 эт.,6169625.0,96100,Время Группа Компаний,р-н Прикубанский,"ул. им. Кирилла Россинского, д. 3/1",...,Время Группа Компаний,https://www.avito.ru/user/0c771d40b5f5e02cde46...,NaN,krasnodar,3,2816012211,2,None,4,6169625


In [132]:
a[a.asd.isna()]

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,...,Продавец,Ссылка на продавца,Завершённых объявлений,city,source_id,ad_id,city_id,district_id,type_id,asd


In [133]:
a

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,...,Продавец,Ссылка на продавца,Завершённых объявлений,city,source_id,ad_id,city_id,district_id,type_id,asd
0,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 17,4 м², 5/5 эт.",1-к. квартира,"17,4 м²",5/5 эт.,2200000.0,126437,NaN,р-н Кировский,"ул. Карпова, 23",...,NaN,NaN,NaN,tomsk,3,2814505758,17,None,3,2200000
1,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 53,5 м², 4/10 эт.",2-к. квартира,"53,5 м²",4/10 эт.,6700000.0,125234,NaN,р-н Октябрьский,"ул. Дизайнеров, 3",...,NaN,NaN,NaN,tomsk,3,2856523926,17,None,4,6700000
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 63 м², 17/18 эт.",2-к. квартира,63 м²,17/18 эт.,6100000.0,96825,NaN,р-н Октябрьский,"микрорайон Солнечный, ул. Герасименко, 3/16",...,Алёна,https://www.avito.ru/user/a09c2534dd270499116e...,95,tomsk,3,3010508456,17,None,4,6100000
3,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,"3-к. квартира, 68 м², 1/9 эт.",3-к. квартира,68 м²,1/9 эт.,5100000.0,75000,NaN,р-н Советский,"Сибирская ул., 102/1",...,Галина,https://www.avito.ru/user/e8f384adaa1ac950015f...,59,tomsk,3,2884326186,17,None,5,5100000
4,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,"Квартира-студия, 20,7 м², 9/10 эт.",Квартира-студия,"20,7 м²",9/10 эт.,2650000.0,128019,NaN,NaN,"пос. Зональная Станция, микрорайон Южные Ворот...",...,NaN,NaN,NaN,tomsk,3,2931555827,17,None,2,2650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114152,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 53,3 м², 15/24 эт.",2-к. квартира,"53,3 м²",15/24 эт.,5882357.0,110363,Группа компаний ИНСИТИ,р-н Прикубанский,"б-р им. Адмирала Пустошкина, д. 11, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2326231151,2,None,4,5882357
114153,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 53,3 м², 20/24 эт.",2-к. квартира,"53,3 м²",20/24 эт.,5882201.0,110360,Группа компаний ИНСИТИ,р-н Прикубанский,"б-р им. Адмирала Пустошкина, д. 11, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2358537090,2,None,4,5882201
114154,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 65,1 м², 7/19 эт.",2-к. квартира,"65,1 м²",7/19 эт.,6184500.0,95000,Группа компаний ИНСИТИ,р-н Прикубанский,"ул. Адмирала Серебрякова, д. 3, корп. 1",...,Группа компаний ИНСИТИ,https://www.avito.ru/user/d1aa4a721290dfdad727...,7916,krasnodar,3,2326806083,2,None,4,6184500
114155,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 15/16 эт.",2-к. квартира,"64,2 м²",15/16 эт.,6169625.0,96100,Время Группа Компаний,р-н Прикубанский,"ул. им. Кирилла Россинского, д. 3/1",...,Время Группа Компаний,https://www.avito.ru/user/0c771d40b5f5e02cde46...,NaN,krasnodar,3,2816012211,2,None,4,6169625


In [134]:
# addr

In [135]:
# создание поля addr из данных avito
# поле должно совпасть с форматом cian
# формат: Томская область; Томск; р-н Кировский; мкр. Преображенский; улица Дзержинского; 36
def avito_addr_from_row(row, city_df):
    # добавление города для совпадения форматов
    try:
        addr = city_df[city_df.url_avito == row['city']]['name'].values[0]
        addr += '; '
    except:
        addr = ''
    # добавляем район, в авито он может быть None
    try:
        if len(row['Район']) == 0:
            pass
        else:
            addr += row['Район']
            addr += '; '
    except:
        pass
    # добавляем улицу, дом, пр. в авито он может быть None
    try:
        for value in row['Улица'].split(', '):
            addr += value
            addr += '; '
    except:
        pass
    # drop duplicates
    addr = addr[:-2]
    if len(addr.split('; ')) != len(set(addr.split('; '))):
        addr = '; '.join(list(dict.fromkeys(addr.split('; '))))
    return addr

In [136]:
a['addr'] = a.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)

In [137]:
# square

In [138]:
def square_from_ploshad(square_value):
    try:
        return float(square_value[:-3].replace(',', '.'))
    except:
        return None

In [139]:
a['square'] = a['Площадь'].apply(lambda x: square_from_ploshad(x))

In [140]:
# floor and house_floors

In [141]:
def floor_floors_from_etazh(etazh_value):
    try:
        return [etazh_value.split('/')[0], etazh_value.split('/')[1].replace(' эт.', '')]
    except:
        return [None, None]

In [142]:
a['floor'],  a['house_floors'] = zip(*a['Этаж'].apply(floor_floors_from_etazh))

In [143]:
# link

In [144]:
a['link'] = a['Ссылка']

In [145]:
# date

In [146]:
files_to_process_avito

['20-03-23 (без дублей).csv']

In [147]:
a['date'] = get_date_from_name(Path(files_to_process_avito[0]).stem)

In [148]:
# status

In [149]:
a['status'] = None

In [150]:
# version

In [151]:
def get_version(engine, today_date, source_id):
    last_version = f"SELECT Max(version) FROM realty WHERE date < '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [152]:
current_source = a.source_id.unique()[0]
current_date = datetime.now().date()
last_version = get_version(sql_engine, current_date, current_source)[0].iloc[0,0]
if last_version == None:
    current_version = 1
else:
    current_version = last_version + 1

In [153]:
a['version'] = current_version

In [154]:
# offer_from
# ВОПРОС - надо ли преобразовать к одному виду с cian

In [155]:
a['offer_from'] = a['Тип продавца']

In [156]:
# status_new

In [157]:
a['status_new'] = 1

In [158]:
# house_id
# нет в текущем алго, оставляю пустым

In [167]:
realty_table = a[['source_id', 'ad_id', 'city_id', 'district_id', 'type_id', 'addr', 'square', 'floor', 'house_floors', 'link', 'date', 'status', 'version', 'offer_from', 'status_new']]

In [172]:
realty_table_1

,source_id,ad_id,city_id,type_id,addr,square,floor,house_floors,link,date,version,offer_from,status_new
0,3,2814505758,17.0,3,Томск; р-н Кировский; ул. Карпова; 23,17.4,5,5,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,2023-03-20,2,Частное лицо,1
1,3,2856523926,17.0,4,Томск; р-н Октябрьский; ул. Дизайнеров; 3,53.5,4,10,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,2,Частное лицо,1
2,3,3010508456,17.0,4,Томск; р-н Октябрьский; микрорайон Солнечный; ...,63.0,17,18,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,2,Агентство,1
3,3,2884326186,17.0,5,Томск; р-н Советский; Сибирская ул.; 102/1,68.0,1,9,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,2023-03-20,2,Агентство,1
4,3,2931555827,17.0,2,Томск; пос. Зональная Станция; микрорайон Южны...,20.7,9,10,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,2023-03-20,2,Частное лицо,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114152,3,2326231151,2.0,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,15,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,2,Застройщик,1
114153,3,2358537090,2.0,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,20,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,2,Застройщик,1
114154,3,2326806083,2.0,4,Краснодар; р-н Прикубанский; ул. Адмирала Сере...,65.1,7,19,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,2,Застройщик,1
114155,3,2816012211,2.0,4,Краснодар; р-н Прикубанский; ул. им. Кирилла Р...,64.2,15,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,2,Агентство,1


# check and load dev realty

In [74]:
realty_table_check = a[['source_id', 'ad_id', 'city_id', 'type_id', 'addr', 'square', 'floor', 'house_floors', 'link', 'date', 'version', 'offer_from', 'status_new']]

In [75]:
a.iloc[realty_table_check[realty_table_check.isna().any(axis=1)].index, :]

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,...,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id


In [347]:
st_time = datetime.now()
realty_table.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
print(len(realty_table), datetime.now() - st_time)

119493 0:01:45.546200


# dev prices

In [199]:
test_1

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
0,3,2814505758,17,None,3,Томск; р-н Кировский; ул. Карпова; 23,17.4,5,5,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,2023-03-20,None,1,Частное лицо,1,2200000.0,126437
1,3,2856523926,17,None,4,Томск; р-н Октябрьский; ул. Дизайнеров; 3,53.5,4,10,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,None,1,Частное лицо,1,6700000.0,125234
2,3,3010508456,17,None,4,Томск; р-н Октябрьский; микрорайон Солнечный; ...,63.0,17,18,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-03-20,None,1,Агентство,1,6100000.0,96825
3,3,2884326186,17,None,5,Томск; р-н Советский; Сибирская ул.; 102/1,68.0,1,9,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,2023-03-20,None,1,Агентство,1,5100000.0,75000
4,3,2931555827,17,None,2,Томск; пос. Зональная Станция; микрорайон Южны...,20.7,9,10,https://www.avito.ru/tomsk/kvartiry/kvartira-s...,2023-03-20,None,1,Частное лицо,1,2650000.0,128019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114152,3,2326231151,2,None,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,15,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,5882357.0,110363
114153,3,2358537090,2,None,4,Краснодар; р-н Прикубанский; б-р им. Адмирала ...,53.3,20,24,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,5882201.0,110360
114154,3,2326806083,2,None,4,Краснодар; р-н Прикубанский; ул. Адмирала Сере...,65.1,7,19,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Застройщик,1,6184500.0,95000
114155,3,2816012211,2,None,4,Краснодар; р-н Прикубанский; ул. им. Кирилла Р...,64.2,15,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-03-20,None,1,Агентство,1,6169625.0,96100


In [190]:
# realty_id

In [191]:
def get_realty_ids(engine, today_date, source_id):
    id_query = f"SELECT link,id FROM realty WHERE date = '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        id_db = pd.read_sql(text(id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        id_db = None
        exc_code = exc.code
    return id_db, exc_code

In [192]:
current_source = a.source_id.unique()[0]
current_date = datetime.now().date()
id_df = get_realty_ids(sql_engine, current_date, current_source)[0]

In [193]:
id_df

,link,id


In [194]:
avito_prices = pd.DataFrame()

In [195]:
avito_prices['realty_id'] = a['Ссылка'].map(id_df.set_index('link')['id'])

In [196]:
# price

In [197]:
avito_prices['price'] = a['Цена']

In [198]:
avito_prices

,realty_id,price
0,NaN,2200000.0
1,NaN,6700000.0
2,NaN,6100000.0
3,NaN,5100000.0
4,NaN,2650000.0
...,...,...
114152,NaN,5882357.0
114153,NaN,5882201.0
114154,NaN,6184500.0
114155,NaN,6169625.0


In [106]:
# price_sqrm

In [107]:
avito_prices['price_sqrm'] = a['Цена за м2'].astype(float)

In [111]:
# date

In [112]:
avito_prices['date'] = datetime.now().date()

In [87]:
# estimate_min estimate_max estimate_med estimate_sqrm diviation skip fill in default values of sql table

In [110]:
# avito_prices[['estimate_min', 'estimate_max', 'estimate_med', 'estimate_sqrm', 'diviation']] = None

In [113]:
# status skip fill in default values of sql table

In [92]:
# avito_prices['status'] = None

In [108]:
# same_cnt same_urls is_actual leftt empty to fill in default values

In [114]:
# avito_prices[['same_cnt', 'same_urls', 'is_actual']] = None

# check and load dev prices

In [115]:
avito_prices

,realty_id,price,price_sqrm,date
0,NaN,4800000,81633.0,2023-03-21
1,NaN,4650000,106651.0,2023-03-21
2,NaN,7700000,85842.0,2023-03-21
3,NaN,4600000,121053.0,2023-03-21
4,NaN,3690000,95596.0,2023-03-21
...,...,...,...,...
119488,NaN,7200000,101408.0,2023-03-21
119489,NaN,6400000,101587.0,2023-03-21
119490,NaN,19350500,236270.0,2023-03-21
119491,NaN,6097000,130000.0,2023-03-21


In [508]:
st_time = datetime.now()
avito_prices.to_sql(name='prices', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
print(len(avito_prices), datetime.now() - st_time)

119493 0:00:24.363198


# cian load

In [7]:
cian_all = pd.read_csv('21-03-23.csv', delimiter=',', encoding='cp1251')

In [8]:
cian_all.drop_duplicates(subset=['link'], keep='last', inplace=True)

# dev realty cian

In [9]:
cian_realty = pd.DataFrame()

In [10]:
# source_id

In [13]:
source_df_dict = source_df.set_index('id').to_dict()['name']

In [14]:
def source_id_from_link_cian(link_value, source_dict):
    try:
        return list(source_dict.keys())[list(source_dict.values()).index(link_value)]
    except:
        return None

In [15]:
cian_realty['source_id'] = cian_all['source'].apply(lambda x: source_id_from_link_cian(x, source_df_dict))

In [16]:
# ad_id

In [17]:
cian_realty['ad_id'] = cian_all['ad_id']

In [18]:
# city_id

In [19]:
city_df_dict = city_df.set_index('id').to_dict()['name']

In [20]:
def city_id_from_link(city_value, city_dict):
    try:
        return int(list(city_dict.keys())[list(city_dict.values()).index(city_value)])
    except Exception as e:
        return None

In [21]:
cian_realty['city_id'] = cian_all['city'].apply(lambda x: city_id_from_link(x, city_df_dict))
# удалить записи где city_id пустой (записи городов, которые не присутствуют в списке городов для парсинга)
ind_to_drop = cian_realty[(cian_realty['city_id'].isna())].index.tolist()
cian_realty.drop(ind_to_drop, inplace=True)
cian_all.drop(ind_to_drop, inplace=True)
cian_realty.city_id = cian_realty.city_id.astype(int)

In [22]:
# district_id пока None

In [23]:
cian_realty['district_id'] = None

In [24]:
# type_id

In [25]:
dict_realty_type = {
    '1': ['Комната'],
    '2': ['Квартира-студия', 'Апартаменты-студия', 'Гостинка', 'Студия'],
    '3': ['1-к. квартира', '1-к. апартаменты', 'Своб. планировка', '1-комнатная', '1 комн', 'Свободная планировка'],
    '4': ['2-к. квартира', '2-к. апартаменты', '2-комнатная', '2 комн'],
    '5': ['3-к. квартира', '3-к. апартаменты', '3-комнатная', '3 комн'],
    '6': ['4-к. квартира', '4-к. апартаменты', '4-комнатная', '4 комн'],
    '7': ['5-к. квартира', '5-к. апартаменты', '5-комнатная', '5 комн'],
    '8': ["6 комнат и более", '6 комн'],
}
# 9 аукцион, доля

In [26]:
def type_id_from_room(room_value, type_dict):
    tid = None
    try:
        for ind, values in type_dict.items():
            if room_value in values:
                tid = ind
                return tid
        if any(elem in room_value for elem in ['Доля', 'Аукцион']):
            # tid = 9 # аукцион
            return None
        elif all(elem in room_value for elem in ['Комната ', ' в ', '-к']):
            tid = 1 # комната в квартире
            return tid
        elif any(elem in room_value for elem in ['6-к.', '7-к.', '8-к.', '9-к.', '10 и более-к.', '10 комнат и больше']):
            tid = 8 # 6 комнат и более
            return tid
        else:
            return None # none
    except:
        return None

In [27]:
cian_realty['type_id'] = cian_all['type'].apply(lambda x: type_id_from_room(x, dict_realty_type))
# удалить записи аукционов и других объяв где не опредлен type_id
ind_to_drop = cian_realty[(cian_realty['type_id'].isna())].index.tolist()
cian_realty.drop(ind_to_drop, inplace=True)
cian_all.drop(ind_to_drop, inplace=True)
cian_realty.city_id = cian_realty.city_id.astype(int)

In [28]:
# addr

In [29]:
cian_realty['addr'] = cian_all['addr']

In [30]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr
0,2,210197059,5,None,5,Краснодарский край; Анапа; улица Горького; 2А
1,2,213575526,5,None,5,Краснодарский край; Анапа; улица Краснозеленых...
2,2,232012292,5,None,2,Краснодарский край; Анапа; улица Ленина; 194
3,2,233965925,5,None,3,Краснодарский край; Анапа; улица Толстого; 81
4,2,239179225,5,None,7,Краснодарский край; Анапа; Владимирская улица;...
...,...,...,...,...,...,...
101775,2,285075402,17,None,3,Томская область; Томск; р-н Советский; проспек...
101776,2,285075572,17,None,3,Томская область; Томск; р-н Советский; улица Ш...
101777,2,285076163,17,None,4,Томская область; Томск; р-н Ленинский; улица Р...
101778,2,285076291,17,None,3,Томская область; Томск; р-н Ленинский; Черемош...


In [31]:
# square

In [32]:
cian_realty['square'] = cian_all['square']

In [33]:
# floor and house_floors

In [34]:
cian_realty['floor'],  cian_realty['house_floors'] = cian_all['floor'], cian_all['house_floors']

In [35]:
# link

In [36]:
cian_realty['link'] = cian_all['link']

In [37]:
# date

In [38]:
cian_realty['date'] = datetime.now().date() # make date from filename format 21-03-2023

In [39]:
# status

In [40]:
cian_realty['status'] = None

In [41]:
# version

In [42]:
def get_version(engine, today_date, source_id):
    last_version = f"SELECT Max(version) FROM realty WHERE date < '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [43]:
current_source = cian_realty.source_id.unique()[0]
current_date = datetime.now().date()
last_version = get_version(sql_engine, current_date, current_source)[0].iloc[0,0]
if last_version == None:
    current_version = 1
else:
    current_version = last_version + 1

In [44]:
current_version

270

In [45]:
cian_realty['version'] = current_version

In [46]:
# offer_from

In [47]:
cian_realty['offer_from'] = cian_all['offer_from']

In [48]:
# status_new

In [49]:
cian_realty['status_new'] = 1

In [50]:
# house_id
# нет в текущем алго, оставляю пустым

In [51]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,None,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,None,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,None,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,None,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,None,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,2,285075402,17,None,3,Томская область; Томск; р-н Советский; проспек...,18.0,5,5,https://tomsk.cian.ru/sale/flat/285075402/,2023-03-24,None,270,Агентство недвижимости,1
101776,2,285075572,17,None,3,Томская область; Томск; р-н Советский; улица Ш...,16.5,2,5,https://tomsk.cian.ru/sale/flat/285075572/,2023-03-24,None,270,Агентство недвижимости,1
101777,2,285076163,17,None,4,Томская область; Томск; р-н Ленинский; улица Р...,48.0,5,6,https://tomsk.cian.ru/sale/flat/285076163/,2023-03-24,None,270,Агентство недвижимости,1
101778,2,285076291,17,None,3,Томская область; Томск; р-н Ленинский; Черемош...,39.0,4,10,https://tomsk.cian.ru/sale/flat/285076291/,2023-03-24,None,270,Агентство недвижимости,1


# districts

In [254]:
cian_realty[cian_realty.index.isin(cian_rm.index)].addr.to_list() == cian_rm.addr.to_list()

True

In [54]:
def get_districts(engine):
    districts_query = f"SELECT DISTINCT id, city_id, name FROM districts"
    try:
        con_obj=engine.connect()
        districts_query_db = pd.read_sql(text(districts_query), con=con_obj)
        con_obj.close()
    except Exception as exc:
        print(exc)
        districts_query_db = None
    return districts_query_db

In [63]:
def district_from_rn_mkrn(realty_row, all_districts, sql_engine):
    # для записи новых районов создадим датафрейм и получим последний id района
    add_districts_df = pd.DataFrame(columns=['city_id', 'name'])
    current_max_id = all_districts.id.max()
    # для работы функции создадим объекты текущего р-н и мкр
    current_rn = None
    current_mkrn = None
    # создадим списо куда будем записывать значения
    list_district_id = []
    for addr_part in realty_row.addr.split(';')[:3]:
        if 'мкр.' in addr_part:
            current_mkrn = addr_part.replace(' мкр. ', '')
        elif 'р-н' in addr_part:
            current_rn = addr_part.replace(' р-н ', '')
        else:
            pass
    if current_rn == None and current_mkrn == None:
        return None
    else:
        district_name = current_mkrn if current_mkrn != None else current_rn
        district_intersection = all_districts[(all_districts.city_id == realty_row.city_id) & (all_districts.name == district_name)]
        if len(district_intersection) == 0:
            print(district_name)
            # обновление данных из districts на случай если дистрикт добавлялся во время исполнения скрипта 
            all_districts_upd = get_districts(sql_engine)
            district_intersection_upd = all_districts_upd[(all_districts_upd.city_id == realty_row.city_id) & (all_districts_upd.name == district_name)]
            current_max_id = int(all_districts_upd.id.max())
            # если в обновленном districts нет данных о районе - добавляем
            if len(district_intersection_upd) == 0: 
                add_districts_df.loc[len(add_districts_df)] = [realty_row.city_id, district_name]
                add_districts_df.to_sql(name='districts', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
                return current_max_id + 1
            # если данные о районе есть, возвращаем id
            else:
                return district_intersection_upd.iloc[0,0]
        else:
            return district_intersection.iloc[0,0]

In [79]:
all_districts = get_districts(sql_engine)

In [80]:
cian_realty.district_id = cian_realty.apply(lambda row: district_from_rn_mkrn(row, all_districts, sql_engine), axis=1)

In [82]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,2,285075402,17,384.0,3,Томская область; Томск; р-н Советский; проспек...,18.0,5,5,https://tomsk.cian.ru/sale/flat/285075402/,2023-03-24,None,270,Агентство недвижимости,1
101776,2,285075572,17,384.0,3,Томская область; Томск; р-н Советский; улица Ш...,16.5,2,5,https://tomsk.cian.ru/sale/flat/285075572/,2023-03-24,None,270,Агентство недвижимости,1
101777,2,285076163,17,394.0,4,Томская область; Томск; р-н Ленинский; улица Р...,48.0,5,6,https://tomsk.cian.ru/sale/flat/285076163/,2023-03-24,None,270,Агентство недвижимости,1
101778,2,285076291,17,506.0,3,Томская область; Томск; р-н Уебский лол; улица...,39.0,4,10,https://tomsk.cian.ru/sale/flat/285076291/,2023-03-24,None,270,Агентство недвижимости,1


In [ ]:
# проверка если есть строковые значения в 

In [252]:
def find_str(x):
    try:
        _ = int(x)
        return False
    except:
        return True

In [253]:
cian_realty[(~cian_realty.district_id.isna()) & (cian_realty.district_id.apply(lambda x: tryx(x)))]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new


In [220]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,2,285075402,17,384.0,3,Томская область; Томск; р-н Советский; проспек...,18.0,5,5,https://tomsk.cian.ru/sale/flat/285075402/,2023-03-24,None,270,Агентство недвижимости,1
101776,2,285075572,17,384.0,3,Томская область; Томск; р-н Советский; улица Ш...,16.5,2,5,https://tomsk.cian.ru/sale/flat/285075572/,2023-03-24,None,270,Агентство недвижимости,1
101777,2,285076163,17,394.0,4,Томская область; Томск; р-н Ленинский; улица Р...,48.0,5,6,https://tomsk.cian.ru/sale/flat/285076163/,2023-03-24,None,270,Агентство недвижимости,1
101778,2,285076291,17,394.0,3,Томская область; Томск; р-н Ленинский; Черемош...,39.0,4,10,https://tomsk.cian.ru/sale/flat/285076291/,2023-03-24,None,270,Агентство недвижимости,1


2023-03-24 16:47:03,158| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [203]:
def test_rn_mkrn(realty_row, all_districts):
    current_rn = None
    current_mkrn = None
    for addr_part in realty_row.addr.split(';')[:3]:
        if 'мкр.' in addr_part:
            current_mkrn = addr_part.replace(' мкр. ', '')
        elif 'р-н' in addr_part:
            current_rn = addr_part.replace(' р-н ', '')
        else:
            pass
    if current_rn == None and current_mkrn == None:
        return None
    else:
        district_name = current_mkrn if current_mkrn != None else current_rn
        print(current_rn)
        print(current_mkrn)
        print(district_name)
        district_intersection = all_districts[(all_districts.city_id == realty_row.city_id) & (all_districts.name == district_name)]
        if len(district_intersection) == 0:
            return district_name
            # add row to new_district_df
        else:
            print('asd')
            return district_intersection.iloc[0,0]

In [206]:
cian_realty.iloc[-1,:]

source_id                                                       2
ad_id                                                   285076712
city_id                                                        17
district_id                                             Советский
type_id                                                         4
addr            Томская область; Томск; р-н Советский; улица К...
square                                                       46.0
floor                                                           4
house_floors                                                    5
link                   https://tomsk.cian.ru/sale/flat/285076712/
date                                                   2023-03-24
status                                                       None
version                                                       270
offer_from                                                Риелтор
status_new                                                      1
Name: 1017

In [204]:
print(test_rn_mkrn(cian_realty.iloc[-1,:], all_districts))

 Советский
None
 Советский
 Советский


In [208]:
all_districts[all_districts.city_id == 17].iloc[2,2]

'Советский'

In [190]:
def tryx(x):
    try:
        _ = int(x)
        return True
    except:
        return False

In [191]:
cian_realty[cian_realty.district_id.apply(lambda x: tryx(x))]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new


In [ ]:
a['floor'],  a['house_floors'] = zip(*a['Этаж'].apply(floor_floors_from_etazh))

In [89]:
cian_rm.addr.apply(lambda x: 'р-н' in x.split(';')[2])

8576      True
8577      True
8578      True
8579      True
8580      True
          ... 
101775    True
101776    True
101777    True
101778    True
101779    True
Name: addr, Length: 66956, dtype: bool

In [90]:
~cian_rm.raion.isna()

8576      True
8577      True
8578      True
8579      True
8580      True
          ... 
101775    True
101776    True
101777    True
101778    True
101779    True
Name: raion, Length: 66956, dtype: bool

In [94]:
cian_rm[(cian_rm.raion.isna()) & (cian_rm.addr.apply(lambda x: 'р-н' in x.split(';')[2]))]

,city,raion,mikroraion,addr


In [100]:
cian_rm[(~cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: 'мкр' in x.split(';')[2]))]

,city,raion,mikroraion,addr
63536,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63537,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63538,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
63539,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63540,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
...,...,...,...,...
68869,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
68870,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
68871,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
68872,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...


In [120]:
def rn_mkrn(addr_str):
    try:
        if 'мкр' in addr_str.split(';')[3]:
            return True
        else:
            return False
    except:
        if 'мкр' in addr_str.split(';')[2]:
            return True
        else:
            return False

In [121]:
'мкр' in cian_rm[(cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: mkr_3rd(x)))].iloc[100,-1].split(';')[3]

True

In [125]:
cian_rm[(cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: mkr_3rd(x)))].iloc[100,-1]

'Краснодарский край; Сочи; р-н Хостинский; Бытха мкр; улица Ясногорская; 16/6к3'

2023-03-24 15:26:41,032| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [76]:
cian_realty.head(5)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,None,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,None,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,None,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,None,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,None,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1


In [ ]:
st_time = datetime.now()
cian_realty.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
print(len(realty_table), datetime.now() - st_time)

# dev prices cian

In [83]:
# realty_id

In [86]:
cian_all

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15


In [87]:
cian_realty.source_id.unique()[0]

2

In [84]:
current_source = cian_realty.source_id.unique()[0]
current_date = datetime.now().date()
id_df = get_realty_ids(sql_engine, current_date, current_source)[0]

NameError: name 'get_realty_ids' is not defined

In [277]:
id_df

,link,id
0,https://krasnodar.cian.ru/sale/flat/268501106/,32201
1,https://anapa.cian.ru/sale/flat/233965925/,32823
2,https://anapa.cian.ru/sale/flat/264443644/,33491
3,https://anapa.cian.ru/sale/flat/264569881/,33638
4,https://anapa.cian.ru/sale/flat/264931277/,33765
...,...,...
32579,https://krasnodar.cian.ru/sale/flat/274097524/,817839
32580,https://krasnodar.cian.ru/sale/flat/274097626/,817840
32581,https://krasnodar.cian.ru/sale/flat/274098241/,817841
32582,https://krasnodar.cian.ru/sale/flat/275302647/,817842


In [350]:
cian_prices = pd.DataFrame()

In [351]:
cian_prices['realty_id'] = cian_realty['link'].map(id_df.set_index('link')['id'])

In [352]:
cian_prices

,realty_id
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
101775,NaN
101776,NaN
101777,NaN
101778,NaN


In [353]:
# price

In [354]:
cian_prices['price'] = cian_all['price']

In [358]:
# price_sqrm

In [359]:
cian_all

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15


In [360]:
cian_prices['price_sqrm'] = cian_all['price_sqrm'].astype(float)

In [361]:
# date

In [362]:
cian_prices['date'] = datetime.now().date() # изменить на имя файла

In [363]:
datetime.now().date()

datetime.date(2023, 3, 22)

In [368]:
datetime.strptime('17-03-23', '%d-%m-%y').date()

datetime.date(2023, 3, 17)

In [372]:
cian_prices

,realty_id,price,price_sqrm,date
0,NaN,25000000,233645.0,2023-03-22
1,NaN,10100000,95283.0,2023-03-22
2,NaN,2100000,105000.0,2023-03-22
3,NaN,4500000,155172.0,2023-03-22
4,NaN,15300000,109286.0,2023-03-22
...,...,...,...,...
101775,NaN,1950000,108333.0,2023-03-22
101776,NaN,2000000,121212.0,2023-03-22
101777,NaN,4350000,90625.0,2023-03-22
101778,NaN,3680000,94359.0,2023-03-22


# write filename to txt

In [376]:
flist[:4]

['C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/01-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/02-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/04-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/05-03-23 (без дублей).csv']

In [443]:
with open('uploaded_files.txt', 'a+') as uploaded_txt:
    for line in flist[:3]:
        uploaded_txt.write(f"{line}\n")
uploaded_txt.close()

In [447]:
try:
    with open('uploaded_files.txt', 'r') as uploaded_txt:
        asd = uploaded_txt.read()
    uploaded_txt.close()
except:
    asd = []

In [448]:
asd

'07-02-2023\n07-02-23 (без дублей)\n08-02-23 (без дублей)\n09-02-23 (без дублей) 2\n09-02-23 (без дублей)\n10-02-23 (без дублей)\n11-02-23 (без дублей)\n12-02-23 (без дублей)\n13-02-23 (без дублей)\n14-02-23 (без дублей)\n15-02-23 (без дублей)\n16-02-23 (без дублей)\n17-02-23 (без дублей)\n18-02-23 (без дублей)\n19-02-23 (без дублей)\n19-02-23\n20-02-23 (без дублей)\n21-02-23 (без дублей)\n22-02-23 (без дублей)\n23-02-23 (без дублей)\n24-02-23 (без дублей)\n25-02-23 (без дублей)\n26-02-23 (без дублей)\n27-02-23 (без дублей)\n28-02-23 (без дублей)\n01-03-23 (без дублей)\n02-03-23 (без дублей)\n04-03-23 (без дублей)\n05-03-23 (без дублей)\n06-03-23 (без дублей)\n07-03-23 (без дублей)\n08-03-23 (без дублей)\n09-03-23 (без дублей)\n10-03-23 (без дублей)\n11-03-23 (без дублей)\n12-03-23 (без дублей)\n13-03-23 (без дублей)\n14-03-23 (без дублей)\n15-03-23 (без дублей)\n16-03-23 (без дублей)\n17-03-23 (без дублей)\n18-03-23 (без дублей)\n19-03-23 (без дублей)\n'

In [449]:
asd.split('\n')[:-1]

['07-02-2023',
 '07-02-23 (без дублей)',
 '08-02-23 (без дублей)',
 '09-02-23 (без дублей) 2',
 '09-02-23 (без дублей)',
 '10-02-23 (без дублей)',
 '11-02-23 (без дублей)',
 '12-02-23 (без дублей)',
 '13-02-23 (без дублей)',
 '14-02-23 (без дублей)',
 '15-02-23 (без дублей)',
 '16-02-23 (без дублей)',
 '17-02-23 (без дублей)',
 '18-02-23 (без дублей)',
 '19-02-23 (без дублей)',
 '19-02-23',
 '20-02-23 (без дублей)',
 '21-02-23 (без дублей)',
 '22-02-23 (без дублей)',
 '23-02-23 (без дублей)',
 '24-02-23 (без дублей)',
 '25-02-23 (без дублей)',
 '26-02-23 (без дублей)',
 '27-02-23 (без дублей)',
 '28-02-23 (без дублей)',
 '01-03-23 (без дублей)',
 '02-03-23 (без дублей)',
 '04-03-23 (без дублей)',
 '05-03-23 (без дублей)',
 '06-03-23 (без дублей)',
 '07-03-23 (без дублей)',
 '08-03-23 (без дублей)',
 '09-03-23 (без дублей)',
 '10-03-23 (без дублей)',
 '11-03-23 (без дублей)',
 '12-03-23 (без дублей)',
 '13-03-23 (без дублей)',
 '14-03-23 (без дублей)',
 '15-03-23 (без дублей)',
 '16-03-

In [166]:
# district_id

In [173]:
district_df['len_addr'] = district_df.addr.apply(lambda x: len(x.split(';')))

In [174]:
district_df['len_addr_coma'] = district_df.addr.apply(lambda x: len(x.split(',')))

In [180]:
district_df.len_addr.value_counts()

6    20113
5    10660
7     6324
1     3610
4     1556
3      135
8      116
2        1
Name: len_addr, dtype: int64

In [185]:
random.randint(0,3610)

1091

In [190]:
district_df[(district_df.city_id == 5)]

,city_id,district_id,addr,len_addr,len_addr_coma
15761,5,NaN,Краснодарский край; Анапа,2,1
15762,5,NaN,Краснодарский край; Анапа; 12-й мкр,3,1
15763,5,NaN,Краснодарский край; Анапа; 12-й мкр; 10,4,1
15764,5,NaN,Краснодарский край; Анапа; 12-й мкр; 11,4,1
15765,5,NaN,Краснодарский край; Анапа; 12-й мкр; 12,4,1
...,...,...,...,...,...
16837,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 47к3,4,1
16838,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к1,4,1
16839,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к2,4,1
16840,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к3,4,1


In [345]:
# district_df[district_df.len_addr == 1].iloc[[random.randint(0,3610) for i in range(50)],:]